In [26]:
#preamble
import os
import pandas as pd
import datetime as dt
from bokeh.models import FactorRange,Legend, LegendItem,ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category20


#get current working directory
cwd = os.getcwd()
#get parent directory
parent = os.path.dirname(cwd)
#get files directory
files = os.path.join(parent, 'files')
#get police department data as pandas dataframe
police = pd.read_csv(os.path.join(files, 'Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240130.csv'))

#take the data from 2010 to 2017
police['Date'] = pd.to_datetime(police['Date'])
#police = police[(police['Date'].dt.year >= 2010) & (police['Date'].dt.year <= 2017)]
police['Hour'] = police['Time'].str.split(':').str[0]



In [7]:
#assault data
assault = police[police['Category'] == 'ASSAULT']
#get the 13 most common assault types
assault_types = assault['Descript'].value_counts().head(13).index

In [69]:
crimes_by_hour = assault.groupby(['Hour', 'Descript']).size().unstack().fillna(0)
#only include the 13 most common assault types
crimes_by_hour = crimes_by_hour[assault_types]

crimes_by_hour.index.name = None
#get all unique hours from the data
hours = crimes_by_hour.index.values
categories = crimes_by_hour.columns.values
#divide each cell by the sum of the column
crimes_by_hour = crimes_by_hour.div(crimes_by_hour.sum(axis=0),axis=1)


In [77]:
#get boke figure
#get boke column data source
source = ColumnDataSource(crimes_by_hour)
p = figure(x_range=FactorRange(factors = [hour for hour in hours]),
           title="Number of Incidents by Hour", width=1200, height=800)


# Add these lines to set the axis labels
p.xaxis.axis_label = 'Hour'
p.yaxis.axis_label = 'Number of Incidents'
colors = Category20[20]

bar  ={}
for i,category in enumerate(categories):
    bar[i] = p.vbar(x='index', top=category, color=colors[i%20], source=source, width=0.9, muted_alpha=0.1, muted = True, alpha=1.0)

# Create a list to hold the legend items
legend_items = [LegendItem(label=category, renderers=[bar[i]]) for i, category in enumerate(categories)]

# Create a new legend with the legend items
legend = Legend(items=legend_items, location="top_left")

# Add the legend to the right of the plot
p.add_layout(legend, 'right')
p.legend.click_policy="mute"
#import show function
from bokeh.io import show
show(p)